<a href="https://colab.research.google.com/github/deepw98/project2/blob/main/Project2P_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp -r /content/drive/MyDrive/fire_detection_few_shot /content/fire_detection_few_shot

In [3]:
import tensorflow as tf
import numpy as np
import os
import random
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, Lambda, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
import pickle


In [4]:
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=target_size)
    img = tf.keras.preprocessing.image.img_to_array(img) / 255.0
    return img

In [5]:
def load_dataset(dataset_path, batch_size=32, img_size=(224, 224)):
    dataset = image_dataset_from_directory(
        dataset_path,
        image_size=img_size,
        batch_size=batch_size
    )
    return dataset


In [6]:
def create_feature_extractor():
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False
    inputs = Input(shape=(224, 224, 3))
    x = base_model(inputs, training=False)
    x = Flatten()(x)
    outputs = Dense(128, activation=None)(x)
    model = Model(inputs, outputs)
    return model


In [7]:
def pairwise_distances(embeddings):
    return tf.norm(tf.expand_dims(embeddings, axis=1) - tf.expand_dims(embeddings, axis=0), axis=-1)


In [8]:
def hard_pair_mining(embeddings, labels):
    distances = pairwise_distances(embeddings)
    pos_mask = tf.equal(tf.expand_dims(labels, axis=1), tf.expand_dims(labels, axis=0))
    neg_mask = tf.logical_not(pos_mask)

    hardest_positive = tf.reduce_max(tf.where(pos_mask, distances, tf.zeros_like(distances)), axis=1)
    hardest_negative = tf.reduce_min(tf.where(neg_mask, distances, tf.fill(tf.shape(distances), tf.float32.max)), axis=1)
    return hardest_positive, hardest_negative


In [9]:
def sample_episode(dataset, n_way=5, k_shot=5, q_queries=5):
    """Samples an episode with N classes, K support samples, and Q query samples."""
    class_indices = list(dataset.class_names)
    selected_classes = random.sample(class_indices, n_way)

    support_set = []
    query_set = []

    for cls in selected_classes:
        cls_images = [img for img, label in dataset if dataset.class_names[label] == cls]
        sampled_images = random.sample(cls_images, k_shot + q_queries)
        support_set.extend(sampled_images[:k_shot])
        query_set.extend(sampled_images[k_shot:])

    return support_set, query_set, selected_classes

In [10]:
def compute_prototypes(support_set, feature_extractor):
    """Computes class prototypes by averaging support embeddings."""
    support_embeddings = feature_extractor(support_set)
    prototypes = []

    for i in range(len(support_set) // len(set(support_set.labels))):
        cls_embeddings = support_embeddings[i * len(set(support_set.labels)):(i+1) * len(set(support_set.labels))]
        prototypes.append(tf.reduce_mean(cls_embeddings, axis=0))

    return tf.stack(prototypes)

In [11]:
def prototypical_loss(y_true, embeddings):
    hardest_positive, hardest_negative = hard_pair_mining(embeddings, y_true)
    loss = tf.maximum(hardest_positive - hardest_negative + 0.2, 0.0)
    return tf.reduce_mean(loss)

In [12]:
def prototypical_loss(prototypes, query_embeddings, query_labels):
    """Computes the loss based on distance of query samples from prototypes."""
    distances = tf.norm(query_embeddings[:, None, :] - prototypes[None, :, :], axis=-1)
    predictions = tf.argmin(distances, axis=-1)
    loss = tf.keras.losses.sparse_categorical_crossentropy(query_labels, predictions)
    return loss

In [13]:
def train_prototypical_network(dataset, feature_extractor, epochs=10, n_way=5, k_shot=5, q_queries=5):
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    for epoch in range(epochs):
        episode_loss = 0

        for _ in range(len(dataset) // (n_way * (k_shot + q_queries))):
            support_set, query_set, selected_classes = sample_episode(dataset, n_way, k_shot, q_queries)

            with tf.GradientTape() as tape:
                prototypes = compute_prototypes(support_set, feature_extractor)
                query_embeddings = feature_extractor(query_set)
                query_labels = [selected_classes.index(cls) for cls in query_set.labels]
                loss = prototypical_loss(prototypes, query_embeddings, query_labels)

            gradients = tape.gradient(loss, feature_extractor.trainable_variables)
            optimizer.apply_gradients(zip(gradients, feature_extractor.trainable_variables))
            episode_loss += loss.numpy()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {episode_loss/len(dataset)}")


In [14]:
# def train_model(dataset_path, epochs=10, save_path="embeddings.pkl"):
#     dataset = load_dataset(dataset_path)
#     feature_extractor = create_feature_extractor()
#     optimizer = Adam(learning_rate=0.001)

#     all_embeddings = {}

#     for epoch in range(epochs):
#         epoch_loss = 0
#         epoch_embeddings = []
#         epoch_labels = []

#         for images, labels in dataset:
#             with tf.GradientTape() as tape:
#                 embeddings = feature_extractor(images)
#                 loss = prototypical_loss(labels, embeddings)

#             gradients = tape.gradient(loss, feature_extractor.trainable_variables)
#             optimizer.apply_gradients(zip(gradients, feature_extractor.trainable_variables))

#             epoch_loss += loss.numpy()
#             epoch_embeddings.append(embeddings.numpy())
#             epoch_labels.append(labels.numpy())

#         print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(dataset)}")

#         all_embeddings[epoch + 1] = {
#             "embeddings": np.concatenate(epoch_embeddings, axis=0),
#             "labels": np.concatenate(epoch_labels, axis=0),
#         }

#     with open(save_path, "wb") as f:
#         pickle.dump(all_embeddings, f)
#     print(f"Embeddings saved to {save_path}")

#     return feature_extractor  # Return trained model

# # Train and get model
# feature_extractor = train_model("/content/fire_detection_few_shot/train")


In [15]:
# train_model("/content/fire_detection_few_shot/train")

In [16]:
# Load dataset
dataset_path = "/content/fire_detection_few_shot/train"
dataset = load_dataset(dataset_path)

# Create feature extractor
feature_extractor = create_feature_extractor()

# Train the prototypical network
train_prototypical_network(dataset, feature_extractor, epochs=10, n_way=5, k_shot=5, q_queries=5)


Found 294 files belonging to 2 classes.
Epoch 1/10, Loss: 0.0
Epoch 2/10, Loss: 0.0
Epoch 3/10, Loss: 0.0
Epoch 4/10, Loss: 0.0
Epoch 5/10, Loss: 0.0
Epoch 6/10, Loss: 0.0
Epoch 7/10, Loss: 0.0
Epoch 8/10, Loss: 0.0
Epoch 9/10, Loss: 0.0
Epoch 10/10, Loss: 0.0


In [17]:
def sample_episode(dataset, n_way=2, k_shot=5, q_queries=5):
    """Samples an episode with dynamic k_shot and q_queries based on available images."""
    class_indices = dataset.class_names
    selected_classes = random.sample(class_indices, min(n_way, len(class_indices)))

    # Convert dataset to a list of (image, label) pairs
    all_images = []
    all_labels = []
    for batch_images, batch_labels in dataset:
        for img, label in zip(batch_images, batch_labels.numpy()):  # Convert labels to NumPy scalars
            all_images.append(img)
            all_labels.append(int(label))  # Ensure labels are integers

    support_set = []
    query_set = []

    for cls in selected_classes:
        cls_index = class_indices.index(cls)
        cls_images = [img for img, lbl in zip(all_images, all_labels) if lbl == cls_index]

        # Ensure enough samples exist
        max_samples = len(cls_images) // 2  # Split into support/query sets
        k_shot = min(k_shot, max_samples)
        q_queries = min(q_queries, max_samples)

        if len(cls_images) < (k_shot + q_queries):
            raise ValueError(f"Class {cls} has only {len(cls_images)} images, but {k_shot + q_queries} are needed.")

        sampled_images = random.sample(cls_images, k_shot + q_queries)
        support_set.extend(sampled_images[:k_shot])
        query_set.extend(sampled_images[k_shot:])

    return support_set, query_set, selected_classes


In [18]:
test_dataset = load_dataset('/content/fire_detection_few_shot/test')
support, query, classes = sample_episode(test_dataset)
print(f"Sampled {len(support)} support images and {len(query)} query images for {len(classes)} classes.")


Found 20 files belonging to 2 classes.
Sampled 10 support images and 10 query images for 2 classes.


In [19]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

def evaluate_prototypical_network(dataset, feature_extractor, n_way=2, k_shot=5, q_queries=5):
    """Evaluates the prototypical network and computes performance metrics."""

    # Sample an episode
    support_set, query_set, selected_classes = sample_episode(dataset, n_way, k_shot, q_queries)

    # Convert images to tensors
    support_images = tf.stack(support_set)
    query_images = tf.stack(query_set)

    # Extract embeddings
    support_embeddings = feature_extractor(support_images)
    query_embeddings = feature_extractor(query_images)

    # Compute class prototypes (mean embedding per class)
    prototypes = []
    for i in range(n_way):
        class_support = support_embeddings[i * k_shot: (i + 1) * k_shot]  # Get support samples for class i
        prototype = tf.reduce_mean(class_support, axis=0)  # Compute mean embedding
        prototypes.append(prototype)

    prototypes = tf.stack(prototypes)  # Shape: (n_way, embedding_dim)

    # Compute Euclidean distances from query embeddings to prototypes
    distances = tf.norm(query_embeddings[:, None, :] - prototypes[None, :, :], axis=-1)  # Shape: (num_queries, n_way)

    # Assign each query to the nearest prototype
    predictions = tf.argmin(distances, axis=-1).numpy()  # Convert tensor to NumPy
    true_labels = np.concatenate([[i] * q_queries for i in range(n_way)])  # True labels based on episode

    # Compute performance metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average="weighted", zero_division=0)
    recall = recall_score(true_labels, predictions, average="weighted", zero_division=0)
    f1 = f1_score(true_labels, predictions, average="weighted", zero_division=0)

    # Print detailed classification report
    print(f"Evaluation Results (n_way={n_way}, k_shot={k_shot}, q_queries={q_queries}):")
    print(f"Accuracy: {accuracy * 100:.2f}%")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")
    print("\nClassification Report:")
    print(classification_report(true_labels, predictions, target_names=selected_classes, zero_division=0))

    return accuracy, precision, recall, f1


In [26]:
# Load dataset
test_dataset = load_dataset('/content/fire_detection_few_shot/test')

# Create feature extractor
feature_extractor = create_feature_extractor()  # Ensure this function is defined

# Evaluate
evaluate_prototypical_network(test_dataset, feature_extractor, n_way=2, k_shot=5, q_queries=5)


Found 20 files belonging to 2 classes.
Evaluation Results (n_way=2, k_shot=5, q_queries=5):
Accuracy: 100.00%
Precision: 1.0000
Recall: 1.0000
F1-score: 1.0000

Classification Report:
              precision    recall  f1-score   support

     No_Fire       1.00      1.00      1.00         5
        Fire       1.00      1.00      1.00         5

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10



(1.0, 1.0, 1.0, 1.0)